In [1]:
# ###===================================================================================================================================================================
# ###===================================================================================================================================================================
# ========================================================================================================================================================
# ================================================================CODE SNIPPETS===========================================================================
# ========================================================================================================================================================
# for a in cm._cmap_names_categorical:
#     display(cm.__dict__[a].resampled(4))
# # ======================================================================================================================================================
# # ======================================================================================================================================================
# for (Event,Station,Metrics,Comp) in OBS_Generator(catalog,ATaCR_Py_DataFolder['Py_DataParentFolder']):
#     display(Event)
# for i,(Event,Station,Metrics,Comp) in zip(range(1),OBS_Generator(catalog,ATaCR_Py_DataFolder['Py_DataParentFolder'])):
#     print(Event)
# # ======================================================================================================================================================
# # ======================================================================================================================================================
# display(catalog)
# print('Compliance band for dataset: ' + str(int(1/fnotch(52))) + '-' + str(int(1/fnotch(5949))) + 's')
# print('-----'*10)
# print('Water Depth min:  ' + str(catalog.Water_Depth_m.min()))
# print('Water Depth max:  ' + str(catalog.Water_Depth_m.max()))
# print('-----'*10)
# print('Sediment Thickness min:  ' + str(catalog.Sediment_Thickness_m.min()))
# print('Sediment Thickness max:  ' + str(catalog.Sediment_Thickness_m.max()))
# print('-----'*10)
# print('Distance to Plate Boundary min:  ' + str(catalog.Distance_to_Plate_Boundary_km.min()))
# print('Distance to Plate Boundary max:  ' + str(catalog.Distance_to_Plate_Boundary_km.max()))
# print('-----'*10)
# print('Distance from Land min:  ' + str(catalog.Distance_from_Land_km.min()))
# print('Distance from Land max:  ' + str(catalog.Distance_from_Land_km.max()))
# print('-----'*10)
# print('Surface Current min:  ' + str(catalog.Surface_Current_ms.min()))
# print('Surface Current max:  ' + str(catalog.Surface_Current_ms.max()))
# print('-----'*10)
# print('# Events mean:  ' + str(int(catalog.n_events.mean()) - int(catalog.n_events.std())))
# print('-----'*10)
# print('Events:  ' + str(len(np.unique(events))))
# print('Depth Km min:  ' + str(np.min(catalog.Depth_KM.min())))
# print('Depth Km max:  ' + str(np.max(catalog.Depth_KM.max())))
# print('-----'*10)
# print('Start min:  ' + str(catalog.Start.min()))
# print('Start max:  ' + str(catalog.End.max()))

In [2]:
# ===================================================================================================
# ============================================ IMPORTS ==============================================
# ===================================================================================================
import sys
from pathlib import Path
project_path = Path('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/ATACR_HPS_Comp')
# sys.path.insert(1,'/Users/charlesh/Documents/Codes/')
# sys.path.insert(0, '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/COMPS')
sys.path.append(str(project_path / 'Packages'))
sys.path.insert(0, str(project_path / 'Packages' / 'ATaCR'))
sys.path.insert(0, str(project_path / 'Packages' / 'CompCode'))
sys.path.insert(0, str(project_path / 'Packages' / 'NoiseCut'))
sys.path.insert(0, str(project_path / 'Packages' / 'ATaCR'/ 'OBStools'))

import requests 
from PIL import Image 
import math
import scipy
import numpy as np
import librosa
import os
import shutil
from scipy.signal import stft, detrend
os.environ['PYDEVD_WARN_SLOW_RESOLVE_TIMEOUT'] = '2'
from obspy import Trace
import librosa.display
import matplotlib.pyplot as plt
import matplotlib.image as img
import matplotlib.gridspec as gridspec
import matplotlib
import sys
import obspy
from obspy.signal import PPSD

from scipy import interpolate

import pickle as pkl
import glob as g
from obspy.clients.fdsn import Client
import datetime
import re
import math
from numpy import linalg as eigen
# import noisecut
import matplotlib.colors as mcolors
import matplotlib.cm as cm2
from scipy.stats import norm
import scipy.stats as stats
from scipy import fft
# import ntk
# from cmcrameri import cm

from scipy.interpolate import RBFInterpolator, InterpolatedUnivariateSpline #<----Experimental

import ObsQA as OBS
# from ObsQA.classes import OBSMetrics as OBSM
# from ObsQA.plots import qtp
from ObsQA import *
# OBSM = OBSMetrics
import obstools as obs
import cmath
from comp_tools import *
from scipy.signal import csd as _csd
import obspy.imaging.cm as cm
from ObsQA.TOOLS.io import *
from obspy.geodetics.base import locations2degrees
from obspy.geodetics.base import degrees2kilometers
sys.path.insert(0, '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/COMPS')
import ObsQA as OBS

In [3]:
# d = []
# e = []
# for evi,ev in enumerate(evaudit[evaudit.nsta>9].iloc):
#     for s in ev.Stations:
#         ella = [ev.ev_lla,ev.ev_lon]
#         slla = [catalog[catalog.Station==s].iloc[0].Latitude,catalog[catalog.Station==s].iloc[0].Longitude]
#         dist = int(degrees2kilometers(locations2degrees(ella[0],ella[1],slla[0],slla[1])))
#         d.append(dist)
#         if dist<30:
#             e.append(evi)

In [4]:
# ---------------------------------------------------------------------------------------------------
# ============================================ FOLDERS ===========================================
# ---------------------------------------------------------------------------------------------------
project_path = Path('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/ATACR_HPS_Comp')
ATaCR_DataFolder = str(project_path / '_DataArchive' / 'ATaCR_Data')
HPS_DataFolder =str(project_path / '_DataArchive' / 'HPS_Data')
NoiseFolder = '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE'
plotfolder = str(project_path / '_FigureArchive' / '_GEN5')
ATaCR_Py_DataFolder = OBS.TOOLS.io.dir_libraries(ATaCR_DataFolder)[1]
dirs = ATaCR_Py_DataFolder
datafolder = ATaCR_Py_DataFolder['Py_DataParentFolder']
eventsfolder = ATaCR_Py_DataFolder['Py_CorrectedTraces']
Folder = Path(plotfolder) / 'MeetingFigs'
Folder.mkdir(exist_ok=True)
# CompFolder = NoiseFolder + '/COMPS/ATaCR_NC'
# MethodsFolder = NoiseFolder + '/METHODS'
# ---------------------------------------------------------------------------------------------------
# ============================================ LOAD DATA ===========================================
# ---------------------------------------------------------------------------------------------------
catalog = pd.read_pickle(eventsfolder + '/event_catalog_updated.pkl')
Station,evi = catalog.iloc[22],3
Event = Station.Events[evi]
# catalog = pd.read_pickle('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS/ATaCR/ATaCR_Python/Metrics/EVENTS/EventMetrics_using_STA_avgTFs.pkl')
# catalog = pd.read_pickle('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS/ATaCR/ATaCR_Python/EVENTS/event_catalog_updated.pkl')
# catalog = pd.read_pickle(eventsfolder + '/sta_catalog_evrecord_set_goodchans_updated.pkl')
# catalog = catalog.drop(index=29)
catalog = pd.read_pickle(eventsfolder + '/sta_catalog_proxima_test.pkl')
# evaudit = ObsQA.io.audit_events(eventsfolder)
evaudit = pd.read_pickle(Path(eventsfolder) / 'event_record_audit.pkl')

def smooth(d,k=10):
        return np.convolve(d, np.ones(k) / k, mode='same')

In [9]:
catalog[catalog.Station=='M08A'].T

,22
Station,M08A
Network,7D
Latitude,44.118698
Longitude,-124.895302
Experiment,CASCADIA INITIATIVE
Instrument_Design,AB
Seismometer,Trillium Compact
Environment,North Pacific
Pressure_Gauge,DPG
Water_Depth_m,126.4


In [ ]:
events = []
_ = [events.extend(c) for c in catalog.Events]
len(np.unique(events))


print('Networks:  ' + str(len(catalog.Network.unique())))
print('             |:| ' + str(catalog.Experiment.unique()))
print('-----'*10)
print('Seismometers :  ' + str(len(catalog.Seismometer.unique())))
print('             |:| ' + str(catalog.Seismometer.unique()))
print('-----'*10)
print('Instrument Designs:  ' + str(len(catalog.Instrument_Design.unique())))
print( '             |:| ' + str(catalog.Instrument_Design.unique()))
print('-----'*10)
print('DPG Pressure Gauges:  ' + str(np.sum(catalog.Pressure_Gauge=='DPG')))
print('APG Pressure Gauges:  ' + str(np.sum(catalog.Pressure_Gauge=='APG')))


In [ ]:
def make_ppsd(self,verbose=False,ppsd_length=None,db_bins=(-200, -50, 1.0),overlap=0.5,skip_on_gaps=True):
    self.ppsd = dict()
    for key in list(self.traces.keys()):
        st = Stream(self.traces[key])
        if not ppsd_length:
            fs = 1/st[0].stats.delta
            # ppsd_length = int(round(2**np.ceil(np.log2(120*(fs))))) / st[0].stats.delta
            ppsd_length = int(round(2**np.ceil(np.log2(26*(fs))))) / st[0].stats.delta
        metadata = {'gain':1., 'sensitivity': 1., 'poles':[1-1j], 'zeros': [1-1j]}
        stats = st[0].stats
        ppsd_prefs = AttribDict()
        ppsd_prefs.skip_on_gaps=skip_on_gaps
        ppsd_prefs.ppsd_length=ppsd_length
        ppsd_prefs.db_bins=db_bins
        ppsd_prefs.overlap=overlap
        ppsd = PPSD(stats = stats,metadata = metadata,**ppsd_prefs)
        success = ppsd.add(st,verbose=verbose)
        if success:
            print(key + ' | Number of psd segments:', len(ppsd.times_processed))
        else:
            print('PPSD Failed')
        self.ppsd[k] = ppsd
    return self
def ppsd_plot(comp,
    # --------Post plot tweaks
    figsize = [15,8],
    eq_lines = ['gray',0.9,'-'],
    suptitle = None,
    average_line = ['k',2,'-.'],
    earthquakes = (5,7,3000),
    period = [1,200],
    show_coverage = False,
    # cmap = obspy.imaging.cm.viridis
    cmap = obspy.imaging.cm.pqlx,
    xaxis_frequency = False,
    append_title = None,
    # --------
    ):
    ppsd = self.ppsd[comp]
    if earthquakes==True:
        earthquakes = (5,7,3000)
    plot_prefs = AttribDict()
    if average_line:
        plot_prefs.show_mean = True
    else:
        plot_prefs.show_mean = False
    plot_prefs.show_coverage=show_coverage
    plot_prefs.show_earthquakes=earthquakes
    plot_prefs.period_lim=period
    plot_prefs.cmap=cmap
    plot_prefs.show=False
    plot_prefs.xaxis_frequency=xaxis_frequency
    # average_line = None
    hevent = ppsd.plot(**plot_prefs)
    _ = hevent.set_figwidth(figsize[0])
    _ = hevent.set_figheight(figsize[1])
    hevent.suptitle(suptitle)
    # Assuming 'hevent' is your figure object
    fig = hevent
    # Access the first axes in the figure (adjust the index if needed)
    ax = fig.axes[0]
    if earthquakes:
        texts = [text for text in ax.get_children() if isinstance(text,plt.Text)][:-3]
        [t.set_position([t.get_position()[0]*(2.1+0.0*ti),t.get_position()[1]*(0.911-0.013*ti)]) for ti,t in enumerate(texts)]
        [t.set_text(t.get_text().replace('\n',' - ')) for t in texts]
        [t.set_rotation(20) for t in texts]
    # Identify line objects
    lines = [line for line in ax.get_children() if isinstance(line, plt.Line2D)]
    stack = 0
    if average_line:
        # averaging line
        lines[0].set_color(average_line[0])  # Change color of the first line to red
        lines[0].set_linewidth(average_line[1])
        lines[0].set_linestyle(average_line[2])
    if plot_prefs.show_mean:
        stack +=1
    # noise model lines
    lines[stack].set_color('red')  # Change color of the first line to red
    lines[stack+1].set_color('red')  # Change color of the second line to blue
    stack+=2
    if plot_prefs.show_earthquakes:
        _ = [l.set_color(eq_lines[0]) for l in lines[stack:]]
        _ = [l.set_linewidth(eq_lines[1]) for l in lines[stack:]]
        _ = [l.set_linestyle(eq_lines[2]) for l in lines[stack:]]

    N = int(len(ppsd.times_processed))
    NS = int(UTCDateTime(ppsd.times_processed[2]) - UTCDateTime(ppsd.times_processed[0]))

    ttl = fig.axes[0].get_title().replace('  ',' | ')
    ttl = ttl.replace('(','').replace(')','')
    ttl = ttl.replace('{N}/{N}'.format(N=N),str(N))
    ttl = ttl.replace('segments', '({NS}s) segments'.format(NS=NS))

    if append_title:
        ttl = ttl + '\n' + append_title
    fig.axes[0].set_title(ttl)
    fig.canvas.draw()
    plt.close('all')
    # Show the updated figure (if not already displayed)
    return self

In [ ]:
# cat = catalog.copy()
#___________________________________________________
#___These stations will not return pressure data____
# problem_stations = [11, 12, 13, 14, 15, 16, 17, 21, 22, 23, 28, 42, 43, 44, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75,
#  76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
# cat = cat.iloc[problem_stations]

# mustang_folder = Path('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/COMPS/FigureArchive/_GEN5/EventRecords/QualityMetrics/Mustang-NoiseSpec')
# db = None
# quality = 'D'
# channel = 'Z'
# problem_stations = mustang_noise_spec_reports(cat=cat,mustang_folder=mustang_folder,channel=channel,db=db,quality=quality)

In [ ]:
# hps_output = Path('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS/NoiseCut/Data/Output')
# evaudit = pd.read_pickle(Path(eventsfolder) / 'event_record_audit.pkl')
# evaudit = evaudit[evaudit.nsta<10]
# # evaudit = evaudit.iloc[119:]
# raws = ['RawZ','Raw1','Raw2','RawP']
# posts = ['PostHPS','PostHPS_H1','PostHPS_H2']
# for evi,ev in enumerate(evaudit.iloc()):
#   event = ev.Event
#   sta = ev.Stations
#   net = ev.Networks
#   if UTCDateTime.strptime(event,format='%Y.%j.%H.%M').hour >= 22:
#     continue

#   for i,(n,s) in enumerate(zip(net,sta)):
#     (hps_output / f'{n}.{s}').mkdir(parents=True,exist_ok=True)
#     output = dict()
#     print(f'{evi+1}/{len(evaudit)} | {i+1}/{len(net)} : {n}.{s}.{event}')
#     Metrics,Comp = get_metrics_comp(n,s,datafolder,event,return_hps=True,return_atacr=False)
#     if len(Comp)==0:
#       continue
#     Raw = Stream()
#     HPS = Stream()
#     for e in raws:
#       Raw+=Comp[e].copy()
#     for e in posts:
#       HPS+=Comp[e].copy()
#     output['Raw'] = [Raw.copy()]
#     output['HPS'] = [HPS.copy()]
#     output = pd.DataFrame(output)
#     file = f'{n}.{s}.{event}.pkl'
#     output.to_pickle(hps_output / f'{n}.{s}' / file)
#     del Raw,HPS,Comp,Metrics

In [ ]:
# NoiseColors = [mcolors.to_hex(m) for m in [cm.__dict__[e].resampled(30).resampled(6).colors for e in ['devon_categorical']][0]]
# [display(c) for c in [cm.__dict__[e].resampled(70).resampled(5) for e in ['nuuk_categorical','devon_categorical','hawaii_categorical','imola_categorical','lapaz_categorical']]]
# np.array([[mcolors.to_hex(c) for c in r] for r in [cm.__dict__[e].resampled(70).resampled(5).colors for e in ['nuuk_categorical','devon_categorical','hawaii_categorical','imola_categorical','lapaz_categorical']]])

In [ ]:
# ____________________________________________________________________________________________
# ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
# |||||||||||||||||||||||||||||||||||||||||||||||||BEGIN PLOT CODES|||||||||||||||||||||||||||
# ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
# ____________________________________________________________________________________________

In [ ]:
def metric_fractional_different(r,A,B):
    a_f,a_coh = A.Coherence(r)
    b_f,b_coh = B.Coherence(r)
    b_coh = interpolate.interp1d(b_f,b_coh)(a_f)
    frac = a_coh / b_coh
    return a_f,frac

In [ ]:
# datafolder = dirs['Py_DataParentFolder']
# nsta_bar = 20
# # evaudit = ObsQA.io.audit_events(eventsfolder)
# evaudit = pd.read_pickle(Path(eventsfolder) / 'event_record_audit.pkl')
# evaudit = evaudit[evaudit.nsta>=nsta_bar]
# # evaudit = evaudit.iloc[100:]
# # evsilike = ['2010.247.08.52','2013.143.17.19']
# # evsilike = ['2010.247.08.52']
# # evsilike = ['2011.242.06.57','2010.246.16.35','2011.191.00.57','2011.236.17.46','2010.163.19.26','2010.096.22.15']
# # evsilike = ['2010.073.08.08','2010.216.07.15'] # '2010.151.19.51','2010.204.22.51'

# # evsi = [np.where(evaudit.Event==d)[0][0] for d in evsilike]
# # evaudit = evaudit.iloc[evsi]
# # evaudit = evaudit.iloc[2:]
# # evaudit = evaudit[evaudit.MW>=7.1]
# # display(evaudit)
# # folder = 'grouped_bands'
# folder = 'separated_bands'
# alumni_poster = Path('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/COMPS/FigureArchive/_GEN5/EventRecords/AlumniPoster/Events')
# ysep_scl = 1.3
# figsize = (14,13)
# bands = [(1/10,1),(1/30,1/10),(1/100,1/30)]
# trim = (10,7200)
# tapers = [3]
# # methods = ['PostATACR','PostHPS']
# methods = ['PostATACR']
# # methods = ['PostHPS']
# channels = ['Z','1','2']
# for correction_method in methods:
#   current_method = correction_method.replace('Post','')
#   coh_comp = correction_method.replace('PostHPS','HPS').replace('PostATACR','ATaCR')
#   if correction_method=='PostHPS':
#     return_hps = True
#   else:
#     return_hps = False
#   if return_hps:
#     return_atacr = False
#   else:
#     return_atacr = True
#   for taper_mode in tapers:
#     OutFolder = Path(plotfolder)
#     SubFolders = Path('EventRecords') / ('Taper_' + str(taper_mode)) / correction_method / folder
#     OutFolder = OutFolder / SubFolders
#     # OutFolder = alumni_poster
#     OutFolder.mkdir(parents=True,exist_ok=True)
#     plt.figure(figsize=(15,5))
#     for evi,ev in enumerate(evaudit.iloc):
#         print('=='*40)
#         mirror_test = mirror_audit(ev.to_frame().T,datafolder=datafolder)[0]
#         mirror_test = np.array(mirror_test[0]) & np.array(mirror_test[1])
#         mirror_test = np.where(mirror_test)[0]
#         if len(mirror_test)==0:
#           print('|| No data mirrors between sets')
#           continue
#         else:
#           print('|| >>> DATASETS MIRROR POPULATION: ' + str(len(mirror_test)))
#         if len(mirror_test)<nsta_bar:
#           print('Insufficient mirrored data (min:' + str(nsta_bar) + ')')
#           continue
#         event = ev.Event
#         stations = ev.Stations
#         networks = ev.Networks.tolist()
#         networks = [networks[i] for i in mirror_test]
#         stations = [stations[i] for i in mirror_test]
#         evdepth = ev.depth
#         post_record = Stream()
#         pre_record = Stream()
#         print('|| [' + str(evi) + '/' + str(len(evaudit)) + '] ' + event + ' | ')
#         print('||---Begin load')
#         # plt.figure(figsize=(15,5))
#         for i,(net,sta) in enumerate(zip(networks,stations)):
#             # if ev.Event=='2010.151.19.51':
#             #     if sta=='C08W':
#             #         _,_= stations.pop(i),networks.pop(i)
#             #         continue
#             #     if sta=='S01W':
#             #         _,_ = stations.pop(i),networks.pop(i)
#             #         continue
#             Metrics,Comp = get_metrics_comp(net,sta,MethodsFolder,event,return_hps=return_hps,return_atacr=return_atacr,events_folder='EVENTS_Taper_' + str(taper_mode))

#             fq,frac = metric_fractional_different('ZP',Metrics['ATaCR'],Metrics['ATaCR'].Noise)
#             if np.sum(frac>=1)>(0.95*len(frac)):
#               #  Metrics,Comp = get_metrics_comp(net,sta,MethodsFolder,event,return_hps=return_hps,return_atacr=return_atacr,events_folder='EVENTS_Taper_' + str(taper_mode))
#                continue
#             s = 2
#             plt.scatter(fq[frac>=1],frac[frac>=1],c='b',s=s,marker='.')
#             plt.scatter(fq[frac<1],frac[frac<1],c='r',s=s,marker='.')
#             plt.axhline(1,linestyle=':',linewidth=0.5,c='k')
#             # plt.xlim([fq.min(),fq.max()])
#             # plt.ylim([frac.min(),frac.max()])
#             if len(Metrics)==0:
#                 _ = stations.pop(i)
#                 _ = networks.pop(i)
#                 continue
#         plt.yscale('log')
#         plt.xscale('log')
#         plt.xlim([fq.min(),fq.max()])
#         plt.ylim([frac.min(),frac.max()])

In [ ]:
# ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
# |||| 1x3 . SEPARATED BANDS . EVENT RECORDS . PLOT CODE |||||
# ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
datafolder = dirs['Py_DataParentFolder']
nsta_bar = 10
evaudit_focus = evaudit.copy()
evaudit_focus = evaudit_focus[evaudit_focus.nsta>=nsta_bar].copy()
# evaudit_focus = evaudit_focus.iloc[100:]
# evsilike = ['2010.247.08.52','2013.143.17.19']
# evsilike = ['2011.242.06.57','2010.246.16.35','2011.191.00.57','2011.236.17.46','2010.163.19.26','2010.096.22.15']
# evsilike = ['2010.073.08.08','2010.216.07.15'] # '2010.151.19.51','2010.204.22.51'
# evsi = [np.where(evaudit_focus.Event==d)[0][0] for d in evsilike]
# evaudit_focus = evaudit_focus.iloc[evsi]
# evaudit_focus = evaudit_focus.iloc[2:]
# evaudit_focus = evaudit_focus[evaudit_focus.MW>=7.1]
# display(evaudit_focus)
# folder = 'grouped_bands'
folder = 'separated_bands'
ysep_scl = 1.3
figsize = (14,13)
bands = [(1/10,1),(1/30,1/10),(1/100,1/30)]
trim = (10,7200)
# methods = ['PostATACR','PostHPS']
methods = ['PostATACR']
# methods = ['PostHPS','PostATACR']
channels = ['Z','1','2']
for correction_method in methods:
  current_method = correction_method
  coh_comp = correction_method.replace('PostHPS','HPS').replace('PostATACR','ATaCR')
  if correction_method=='PostHPS':
    return_hps = True
  else:
    return_hps = False
  if return_hps:
    return_atacr = False
  else:
    return_atacr = True
  # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
  OutFolder = Path(plotfolder)
  SubFolders = Path('EventRecords') / correction_method / folder
  OutFolder = OutFolder / SubFolders
  OutFolder.mkdir(parents=True,exist_ok=True)
  for evi,ev in enumerate(evaudit_focus.iloc):
      print('=='*40)
      mirror_test = mirror_audit(ev.to_frame().T,datafolder=datafolder)[0]
      mirror_test = np.array(mirror_test[0]) & np.array(mirror_test[1])
      mirror_test = np.where(mirror_test)[0]
      if len(mirror_test)==0:
        print('|| No data mirrors between sets')
        continue
      else:
        print('|| >>> DATASETS MIRROR POPULATION: ' + str(len(mirror_test)))
      if len(mirror_test)<nsta_bar:
        print('Insufficient mirrored data (min:' + str(nsta_bar) + ')')
        continue
      event = ev.Event
      stations = ev.Stations
      networks = ev.Networks.tolist()
      networks = [networks[i] for i in mirror_test]
      stations = [stations[i] for i in mirror_test]
      evdepth = ev.depth
      post_record = Stream()
      pre_record = Stream()
      print('|| [' + str(evi) + '/' + str(len(evaudit_focus)) + '] ' + event + ' | ')
      print('||---Begin load')
      for i,(net,sta) in enumerate(zip(networks,stations)):
        if ev.Event=='2010.151.19.51':
          if sta=='C08W':
            _ = stations.pop(i)
            _ = networks.pop(i)
            continue
          if sta=='S01W':
            _ = stations.pop(i)
            _ = networks.pop(i)
            continue
        # try:
        Metrics,Comp = get_metrics_comp(net,sta,MethodsFolder,event,return_hps=return_hps,return_atacr=return_atacr,events_folder='EVENTS')
        if len(Metrics)==0:
            _ = stations.pop(i)
            _ = networks.pop(i)
            continue
        # except:
        #   _ = stations.pop(i)
        #   _ = networks.pop(i)
        #   continue
        post_record += Comp[current_method].copy()
        pre_record += Comp['RawZ'].copy()
        del Metrics
        del Comp
        if len(post_record)==0:
          continue
      print('||---Load complete')
      phases = ('P','S','SKS','PKiKP','SKiKS','SKSSKS',)
      # phases = ('P','S',)
      # phases=('ttall',)
      evstream = post_record.copy()
      evstream_back = pre_record.copy()
      facecolor=('b','r')
      title = event
      sortindex = None
      normscale = 0.7
      residual_fraction = 0.5
      for chan in channels:
        pre_record_chan = pre_record.select(channel='*'+chan).copy()
        post_record_chan = post_record.select(channel='*'+chan).copy()
        for bandi,(band,s) in enumerate(zip(bands,[[1,1],[1,1],[1,1]])):
          band_sec = np.sort([1/b for b in band])
          print('|| Plotting band: ' + str(band_sec[0]) + ' to ' + str(band_sec[-1]) + 's' + ' | channel:H' + chan)
          fig, axes = plt.subplots(nrows=1, ncols=2,figsize=figsize,layout='constrained',squeeze=False,sharey='all',sharex='all')
          # fig, axes = plt.subplots(nrows=1, ncols=3,figsize=figsize,layout='constrained',squeeze=False,sharey='all',sharex='all')
          # norms='postset'
          norms = 'trace'
          # norms = 'col'
          # norms = np.array([[abs(c.data).max() for c in s] for s in [pre_record,post_record]]).T.max(axis=1).tolist() #global max norm
          # norms = np.array([[abs(c.data).max() for c in s] for s in [pre_record]]).T.max(axis=1).tolist() #pre max norm
          # norms = np.array([[abs(c.data).max() for c in s] for s in [post_record]]).T.max(axis=1).tolist() #post max norm. >This works well at >m6.5
          for record_i,evstream in enumerate([pre_record_chan,post_record_chan]):
            # File = 'm' + str(ev.MW) + '_z' + str(ev.depth) + 'km' + '_' + event + '_' + str(band_sec[0]) + 'to' + str(band_sec[-1]) + 's_' + correction_method.replace('Post','') + '_T'
            File = 'm' + str(ev.MW) + '_z' + str(ev.depth) + 'km' + '_' + event + '_' + str(band_sec[0]) + 'to' + str(band_sec[-1]) + 's_' + correction_method.replace('Post','')
            fig_title = 'm' + str(ev.MW) + '_z' + str(ev.depth) + 'km' + '_' + '-'.join(event.split('.')[:2]) + ' ' + ':'.join(event.split('.')[2:]) + '_' + str(band_sec[0]) + 'to' + str(band_sec[-1]) + 's_Corrected using ' + correction_method.replace('Post','')
            ax = axes[0,record_i]
            # linewidth = [0.05,0.05,0.05][bandi]
            linewidth = [0.2,0.2,0.2][bandi]
            ax_title = ['{} Pre-Correction | Channel:{} \n '.format(current_method,chan),'{} Post-Correction | Channel:{} \n '.format(current_method,chan)][record_i]
            title = File.replace('_',' | ').replace('z','z: ').replace('m','mag: m')
            # ---------------
            # More reasonable black and white plots:
            ax = event_record_plot(evstream=evstream,evstream_back=None,norm=norms,scales=s,linewidth=linewidth,figsize=figsize,band=band,trim=trim,facecolor=facecolor,evdepth=evdepth,phases=phases,title=title,sortindex = sortindex,ax=ax,normscale=normscale,residual_fraction=residual_fraction)
            # Use this for those ugly fill mode plots:
            # ax = event_record_plot(evstream=evstream,evstream_back=None,norm=norm,scales=s,linewidth=linewidth,figsize=figsize,band=band,trim=trim,facecolor=facecolor,evdepth=evdepth,phases=phases,title=title,sortindex = sortindex,ax=ax,normscale=normscale,residual_fraction=residual_fraction)
            # ---------------

            ax.set_title(ax_title,fontweight='bold')          
# --------------------------------------------- Switch between grouped-band and separated bands plots
          # ##### Separated bands
          save_format = 'png'
          fig.suptitle(fig_title.replace('_',' | ').replace('to',' to '),fontweight='bold',fontsize=15)
          s_mag,s_depth,s_chan,s_bands,s_method = ['m' + str(ev.MW), 'z' + str(int(ev.depth)) + 'km' , 'H' + chan , 'to'.join([str(int(b)) for b in band_sec]) + 's',correction_method.replace('Post','')]
          tags = [s_chan,s_bands,s_method,s_mag+s_depth,event]
          File = '_'.join(tags) + '.{}'.format(save_format)
          outfile = OutFolder / s_chan / File
          # print('{} saved to {}'.format(File,OutFolder))
          outfile.parent.mkdir(exist_ok=True,parents=True)
          save_tight(outfile,format=save_format,dpi=200)
          # fig.savefig(outfile,format=save_format,dpi=400)
      # ##### Grouped bands
      # fig.suptitle(File.replace('_',' | ').replace('to',' to '),fontweight='bold',fontsize=15)
      # save_tight(OutFolder / (File + '.eps' ),dpi=600)
# --------------------------------------------
          plt.close('all')
  print('000'*30)
  print(correction_method + ' ||---EV RECORDS COMPLETE---||')
  print('000'*30)

In [ ]:
# nsta_bar = 10
# # |||||||||||||||||||||||||||||||||||||||||||||||||||
# # |||||| Nx2 . COHERENCE RECORDS . PLOT CODE ||||||||
# # |||||||||||||||||||||||||||||||||||||||||||||||||||
# # evaudit = ObsQA.io.audit_events(eventsfolder)
# evaudit = pd.read_pickle(Path(eventsfolder) / 'event_record_audit.pkl')
# evaudit = evaudit[evaudit.nsta>=nsta_bar]

# # evsilike = ['2011.242.06.57','2010.246.16.35','2011.191.00.57','2011.236.17.46','2010.163.19.26','2010.096.22.15']
# # evsilike = ['2011.191.00.57','2010.096.22.15']
# # evsi = [np.where(evaudit.Event==d)[0][0] for d in evsilike]
# # evaudit = evaudit.iloc[evsi]

# # evaudit = evaudit[evaudit.MW>=7.1]
# display(evaudit)
# tapers = [0]
# # methods = ['PostATACR','PostHPS']
# # methods = ['PostATACR']
# methods = ['PostHPS','PostATACR']
# ysep_scl = 1.3
# figsize = (10,10)
# return_noise = True
# for correction_method in methods:
#   coh_comp = correction_method.replace('PostHPS','HPS').replace('PostATACR','ATaCR')
#   if correction_method=='PostHPS':
#     return_hps = True
#     return_atacr = False
#   else:
#     return_hps = False
#     return_atacr = True
  
#   OutFolder = Path(plotfolder)
#   SubFolders = Path('EventRecords') / correction_method / 'coherence'
#   OutFolder = OutFolder / SubFolders
#   OutFolder.mkdir(parents=True,exist_ok=True)
#   for evi,ev in enumerate(evaudit.iloc):
#       event = ev.Event
#       File = 'm' + str(ev.MW) + '_z' + str(ev.depth) + 'km' + '_' + event + '_' + correction_method.replace('Post','') + '_COH'
#       title = File.replace('_',' | ').replace('z','z: ').replace('m','mag: m')

#       mirror_test = mirror_audit(ev.to_frame().T,datafolder=datafolder)[0]
#       mirror_test = np.array(mirror_test[0]) & np.array(mirror_test[1])
#       mirror_test = np.where(mirror_test)[0]
#       if len(mirror_test)==0:
#         print('|| No data mirrors between sets')
#         continue
#       else:
#         print('|| >>> DATASETS MIRROR POPULATION: ' + str(len(mirror_test)))
#       if len(mirror_test)<nsta_bar:
#         print('Insufficient mirrored data (min:' + str(nsta_bar) + ')')
#         continue
#       event = ev.Event
#       stations = ev.Stations
#       networks = ev.Networks.tolist()
#       networks = [networks[i] for i in mirror_test]
#       stations = [stations[i] for i in mirror_test]
#       evdepth = ev.depth
#       post_record = Stream()
#       pre_record = Stream()
#       Metrics = []
#       print('=='*25)
#       print('[' + str(evi) + '/' + str(len(evaudit)) + '] ' + File)
#       print('|| [' + str(evi) + '/' + str(len(evaudit)) + '] ' + event + ' | ')
#       print('||---Begin load')
#       for i,(net,sta) in enumerate(zip(networks,stations)):
#         try:
#           M,Comp = get_metrics_comp(net,sta,MethodsFolder,event,return_hps=return_hps,return_atacr=return_atacr,events_folder='EVENTS')
#           M['Noise'] = get_Noise(dirs['Py_DataParentFolder'],net,sta,'sta')['Noise']
#           Metrics.append(M.copy())
#         except:
#           _ = stations.pop(i)
#           _ = networks.pop(i)
#           continue
#         post_record += Comp[correction_method].copy()
#         pre_record += Comp['RawZ'].copy()
#         del Comp
#       if len(Metrics)==0:
#         continue
#       evstream = post_record.copy()
#       evstream_back = pre_record.copy()
#       title = event
#       sortindex = list(np.argsort([np.abs(st.stats.sac.stel*1000) for st in evstream]))
#       sortindex.reverse()
#       Metrics = [Metrics[s] for s in sortindex]
#       fig, axes = plt.subplots(nrows=2, ncols=2,figsize=figsize,layout='constrained',squeeze=True,sharey='all',sharex='all')
#       axes = axes.flatten()
#       for ci,cmp in enumerate(['ZP','ZZ','1Z','2Z']):
#         ax = axes[ci]
#   # -------------
#         if cmp=='ZP':
#           [ax.scatter(M['Noise'].Coherence(cmp)[0],M['Noise'].Coherence(cmp)[1] + ysep*ysep_scl,c='gray',s=1) for ysep,M in enumerate(Metrics)]
#           [ax.plot(M['Raw'].Coherence(cmp)[0],M['Raw'].Coherence(cmp)[1] + ysep*ysep_scl,c='b',linewidth=0.7) for ysep,M in enumerate(Metrics)]
#         if cmp=='ZP':
#           [ax.plot(M[coh_comp].Coherence(cmp)[0],M[coh_comp].Coherence(cmp)[1] + ysep*ysep_scl,c='r',linewidth=0.1) for ysep,M in enumerate(Metrics)]
#         else:
#           [ax.plot(M[coh_comp].Coherence(cmp)[0],M[coh_comp].Coherence(cmp)[1] + ysep*ysep_scl,c='r',linewidth=0.8) for ysep,M in enumerate(Metrics)]
#   # -------------
#         [ax.plot(M[coh_comp].Coherence(cmp)[0],M[coh_comp].Coherence(cmp)[1]*0 + ysep*ysep_scl,c='k',linewidth=0.1) for ysep,M in enumerate(Metrics)]
#         fn = [fnotch(np.round(1000*abs(M['Raw'].traces.select(channel='*Z')[0].stats.sac.stel))) for M in Metrics]
#         yticks = [ysep*ysep_scl for ysep,_ in enumerate(Metrics)]
#         [ax.plot([f,f],[y,y+1],c='k',linewidth=0.4) for f,y in zip(fn,yticks)]
#         [ax.text(f,y+1,'fn',horizontalalignment='center',fontsize=7,fontweight='bold') for f,y in zip(fn,yticks)]
#         yticklabels = [str(round(1000*abs(M['Raw'].traces.select(channel='*Z')[0].stats.sac.stel))) + 'm [' + str( M['Raw'].traces.select(channel='*Z')[0].stats.network) + '] ' + str( M['Raw'].traces.select(channel='*Z')[0].stats.station) for M in Metrics]
#         ax.set_xscale('log')
#         f = Metrics[0][coh_comp].Coherence(cmp)[0]
#         ax.set_xlim(f[1],f[-1])
#         ax_title = cmp + ' Coherence'
#         ax.set_title(ax_title,fontweight='bold')
#         ax.set_yticks(yticks)
#         ax.set_yticklabels(yticklabels)
#         ax.set_ylim(yticks[0],yticks[-1]+1.3)
#       fig.suptitle(File.replace('_',' | '),fontweight='bold',fontsize=15)
#       save_format = 'png'
#       # fig.suptitle(fig_title.replace('_',' | ').replace('to',' to '),fontweight='bold',fontsize=15)
#       # File = File
#       outfile = OutFolder / (File + '.' + save_format)
#       save_tight(outfile,format=save_format,dpi=500)
#       # (OutFolder / 'PNG').mkdir(parents=True,exist_ok=True)
#       # fig.savefig(OutFolder / 'PNG' / (File + '.png'),format='png',dpi=900)
#       plt.close('all')
#   print('000'*30)
#   print(correction_method + ' ||---COHERENCE RECORDS COMPLETE---||')
#   print('000'*30)
#         # || 26min to complete

In [ ]:
# # ||||||||| Some old HPS Spectrogram Plot Code |||||||||
# # ||||||||||||||||||||||||||||||||||||||||||||||||||||||
# datafolder = dirs['Py_DataParentFolder']
# # stas = ['X01','M07A','J59C','G03D','M08A'] #I only have time to print out a few. Choose these.
# sta = None
# # sta = "G03A"
# # sta = "M08A"
# # sta = "LA34"
# evireq = None
# # events_folder = 'EVENTS'
# events_folder = 'EVENTS'
# # sta = ['M08A']
# #### -----
# spectrogram_win_len = 200
# bands = [[1,1/10],[1/10,1/30],[1/30,1/100],'Spectrogram']
# nrows,ncols = 4,2
# TauP = True
# # bg_tr_color = '#cbc6f4'
# bg_tr_color = '#3266a7'
# Folder = Path(plotfolder) / 'MeetingFigs' / events_folder
# Folder.mkdir(exist_ok=True)
# (Folder / 'HPS_Spectrograms').mkdir(exist_ok=True)
# nsta_bar = 1
# # evaudit = ObsQA.io.audit_events(eventsfolder)
# evaudit = pd.read_pickle(Path(eventsfolder) / 'event_record_audit.pkl')
# # evaudit = evaudit[evaudit.nsta>=nsta_bar]
# evsilike = ['2010.151.19.51']
# # custom = ['YL','PL68']

# evsi = [np.where(evaudit.Event==d)[0][0] for d in evsilike]
# evaudit = evaudit.iloc[evsi]

# # evaudit = evaudit[evaudit.MW>=7.1]
# display(evaudit)
# tapers = [3]
# # methods = ['PostATACR','PostHPS']
# # methods = ['PostATACR']
# methods = ['PostHPS']
# ysep_scl = 1.3
# figsize = (10,10)
# save_format = 'svg'
# for comp_col,correction_method in enumerate(methods):
#   coh_comp = correction_method.replace('PostHPS','HPS').replace('PostATACR','ATaCR')
#   if correction_method=='PostHPS':
#     return_hps = True
#     return_atacr = False
#   else:
#     return_hps = False
#     return_atacr = True
#   for taper_mode in tapers:
#     OutFolder = Path(plotfolder)
#     SubFolders = Path('EventRecords') / ('Taper_' + str(taper_mode)) / correction_method / (correction_method.lower().replace('post','') + '_spectrograms')
#     OutFolder = OutFolder / SubFolders
#     OutFolder = Path('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/COMPS/FigureArchive/_GEN5/EventRecords/AlumniPoster/Specs')
#     OutFolder.mkdir(parents=True,exist_ok=True)
#     for evi,ev in enumerate(evaudit.iloc):
#         event = ev.Event
#         File = 'm' + str(ev.MW) + '_z' + str(ev.depth) + 'km' + '_' + event + '_' + correction_method.replace('Post','') + '_T' + str(taper_mode) + '_Spec'
#         title = File.replace('_',' | ').replace('z','z: ').replace('m','mag: m')

#         mirror_test = mirror_audit(ev.to_frame().T,datafolder=datafolder)[0]
#         mirror_test = np.array(mirror_test[0]) & np.array(mirror_test[1])
#         mirror_test = np.where(mirror_test)[0]
#         if len(mirror_test)==0:
#           print('|| No data mirrors between sets')
#           continue
#         else:
#           print('|| >>> DATASETS MIRROR POPULATION: ' + str(len(mirror_test)))
#         if len(mirror_test)<nsta_bar:
#           print('Insufficient mirrored data (min:' + str(nsta_bar) + ')')
#           continue
#         event = ev.Event
#         stations = ev.Stations
#         networks = ev.Networks.tolist()
#         networks = [networks[i] for i in mirror_test]
#         stations = [stations[i] for i in mirror_test]
# # -----------------------
#         # networks = custom[0]
#         # stations = custom[1]
#         Metrics = []
#         post_record = Stream()
#         pre_record = Stream()
#         for i,(net,sta) in enumerate(zip(networks,stations)):
#           # try:
#           M,Comp = get_metrics_comp(net,sta,datafolder,event,return_hps=return_hps,return_atacr=return_atacr,events_folder='EVENTS_Taper_' + str(taper_mode))
#           if len(M)==0:
#             continue
#           M['Noise'] = get_Noise(dirs['Py_DataParentFolder'],net,sta,'sta')['Noise']
#           # Metrics.append(M.copy())
#           Metrics = M
#           # except:
#           #   _ = stations.pop(i)
#           #   _ = networks.pop(i)
#           #   continue
#           post_record += Comp[correction_method].copy()
#           pre_record += Comp['RawZ'].copy()
#           # Metrics = Metrics[0]
#           Station = catalog[catalog.StaName==(net + '.' + sta)].iloc[0]
#           label = None
#           phases=('P','S','SKS','PKiKP','SKiKS','SKSSKS',)
#           OutPath = OutFolder
#           fig, axes = plt.subplots(nrows=nrows, ncols=ncols,figsize=(22,12),height_ratios=[1,1,1,3],width_ratios=[1,1],layout='constrained',squeeze=False,sharey='row',sharex='row')
#           vmin,vmax = None,None
#           ylabel = True
      
#           File = Station.StaName + '.' + '.'.join(event.split('.')[0:2]) + '_' + correction_method.replace('Post','')
#           fn = np.round(fnotch(Station.Water_Depth_m)*1000)/1000
#           for band_row,band in enumerate(bands):
#             for coli,prepost in enumerate(['Raw',coh_comp]):
#               if coli==1:
#                 lg_on = True
#                 cbar_on = True
#               else:
#                 lg_on = False
#                 cbar_on = False
#               ax = axes[band_row,coli]
#               label = correction_method.replace('ATaCR_HPS','ATaCR & HPS')
#               label = prepost
#               if band_row==0:
#                   ttl = '' + Station.StaName + ' | Station Depth: ' + str(int(Station.Water_Depth_m)) + 'm\n' + event + ''
#                   fig.suptitle(ttl,fontweight='bold',y=1.06)
#               if band!='Spectrogram':
#                   STA_LLAZ = [Station.Latitude,Station.Longitude,Station.Water_Depth_m/1000]
#                   event_LLAZ = [ev.ev_lla,ev.ev_lon,ev.depth]
#                   trim = (UTCDateTime.strptime(event,format='%Y.%j.%H.%M'),UTCDateTime.strptime(event,format='%Y.%j.%H.%M')+7200)
#                   ax = Metrics[prepost].plottrace(ax=ax,component='Z',band=band,color='k',lg=lg_on,st_llaz = STA_LLAZ,label=label,ev_llaz=event_LLAZ,phases=phases,trim=trim)
#                   if coli==0:
#                     band
#                     ax.set_ylabel(str(int(1/band[0])) + 's to ' + str(int(1/band[1])) + 's',fontweight='bold')
#               elif band=='Spectrogram':
#                   yscale = 'log'
#                   pc,ax = Metrics[prepost].spectrogram(plot=True,r='Z',fig=fig,ax=ax,window=spectrogram_win_len,vmin=vmin,vmax=vmax,ylabel=ylabel,cbar_on=cbar_on,yscale=yscale)
#                   # ax.set_yscale('log')
#                   if coli==0:
#                       vmin,vmax = pc.get_clim()
#                       # ylabel = False
#                   else:
#                     ax.set_ylabel('')
#           save_tight(str(OutPath / (File + '.' + save_format)),format=save_format)
#           (OutPath / 'PNG').mkdir(parents=True,exist_ok=True)
#           save_tight(str(OutPath / 'PNG' / (File + '_lowres.png')),format='png',dpi=600)
#           print('[' + str(evi+1) + '/' + str(len(evaudit)) + '][' + str(i+1) + '/' + str(len(stations)) + '] Save complete ' + File)
#           plt.close('all')
# print('****'*20)
# print('------|||| SPECTROGRAMS COMPLETE ||||------')
# print('****'*20)

In [ ]:
# def _next_pow2(n):
#     return int(round(2**np.ceil(np.log2(n))))
# def _valid_win_length_samples(win_length_samples, win_length, sampling_rate):
#     if win_length_samples is None and win_length is None:
#         # fully automatic window length
#         win_length_samples = _next_pow2(120*sampling_rate)
#     elif win_length_samples is None and win_length is not None:
#         win_length_samples = _next_pow2(win_length*sampling_rate)
#     elif win_length_samples is not None and win_length_samples is not None:
#         raise ValueError(
#             'Parameters win_length and win_length_samples are mutually '
#             'exclusive.')
#     elif win_length_samples is not None and win_length is None:
#         # check win_length_samples is a power of 2
#         win_length_samples = int(win_length_samples)
#         if win_length_samples != _next_pow2(win_length_samples):
#             raise ValueError(
#                 'Parameter win_length_samples must be a power of 2.')
#     return win_length_samples

# win_length_samples = _valid_win_length_samples(win_length_samples, win_length, trace.stats.sampling_rate)

# hop_length = win_length_samples // 4
# n_fft = win_length_samples

# # Compute the spectrogram amplitude and phase
# S_full, phase = librosa.magphase(librosa.stft(x,n_fft=n_fft,hop_length=hop_length,win_length=win_length_samples))

In [ ]:
# # ||||||||||||||||||||||||||||||||||||||
# # ||||||||||||||| STEM/LEAF PLOTS ||||||
# # ||||||||||||||||||||||||||||||||||||||
# cat = catalog.copy()
# taper_mode = 0
# return_noise = True
# return_hps = False
# events_folder = 'EVENTS_Taper_' + str(taper_mode)
# figsize = (20,14)
# coh_comps = ['ATaCR_ZZ','Noise_ZP','Raw_ZP','ATaCR_ZP','Raw_1Z','ATaCR_1Z','Raw_2Z','ATaCR_2Z']
# nrows = len(coh_comps) + 1
# fig, axes = plt.subplots(nrows=nrows, ncols=1,figsize=figsize,layout='constrained',squeeze=False,sharey='all',sharex='all')
# for stai,Station in enumerate(cat.iloc):
#   Metrics = []
#   for evi,event in enumerate(Station.Events):
#     notify = Station.StaName + ' ' + str(stai+1) + '/' + str(len(cat)) + ' || ' + str(evi+1) + '/' + str(len(Station.Events))
#     try:
#       M,Comp = get_metrics_comp(Station.Network,Station.Station,datafolder,event,return_hps=return_hps,return_noise=return_noise,events_folder=events_folder)
#       Metrics.append(M.copy())
#       print(notify)
#     except:
#       print(notify + ' | 404 Error')
#       continue
#   X = [stai]
#   stats = metric_stats(Metrics)
#   pre_stalta = stats['log_rms_stalta'][:,0]
#   post_stalta = stats['log_rms_stalta'][:,1]
#   ax = axes[0,0]
#   metric_title = 'STA/LTA'
#   for pi,p in enumerate([pre_stalta,post_stalta]):
#     if stai==0:
#       label = ['Pre','Post'][pi]
#     else:
#       label = None
#     Y = [p.mean()]
#     c = ['b','r'][pi]
#     ls = ['-','--'][pi]
#     minmax = np.atleast_2d(np.array([p.min(),p.max()])).T
#     ax.scatter(X,Y,c=c,s=800,marker='_',label=label)
#     ax.scatter(X,Y-p.std(),c=c,s=100,marker='_')
#     ax.scatter(X,Y+p.std(),c=c,s=100,marker='_')
#     # ax.errorbar(X,Y,yerr=minmax,ecolor=c,ls=ls)
#     ax.vlines(X,minmax[0],minmax[1],colors = c,linestyles=ls,linewidth=3)
#     # ax.errorbar(X,Y,yerr=p.std(), ecolor='r',capthick=0)
#   ax.set_title(metric_title + '\n Min/Max : Vertical Lines ' + ' '*10 + 'σ : Short Horiz. Lines' + ' '*10 + 'μ : Long Horiz. Lines',fontweight='bold')
#   ax.set_ylim(0)
#   ax.legend(ncol=2,loc='upper left')
#   band_colors = '#901f62','#588a7d','#284ea5'
#   bands = [[1/100,1/30],[1/30,1/10],[1/10,1]]
#   bands = 1/np.sort(bands)
#   for axi,coh in enumerate(coh_comps):
#     ax = axes[axi+1,0]
#     [ax.stem(X,np.mean((stats['coh_mu_rms_bands'][coh][:,bi])**2)**0.5,markerfmt=band_colors[bi]) for bi in range(len(bands))]
#     ax.set_title(coh.replace('_','-') + ' Coherence RMS')
#     if stai==0:
#       if axi==0:
#         [ax.stem(X,np.mean((stats['coh_mu_rms_bands'][coh][:,bi])**2)**0.5,markerfmt=band_colors[bi],label=str(bands[bi][1]) + '-' + str(bands[bi][0]) + 's',linefmt=band_colors[bi],basefmt='w') for bi in range(len(bands))]
#     ax.legend(ncol=3,loc='upper left')
#     ax.set_ylim(0)
# ax = axes[-1,0]
# xticks = [xsep for xsep,_ in enumerate(cat.iloc)]
# xticklabels = [c.StaName for c in cat.iloc]
# ax.set_xticks(xticks)
# ax.set_xticklabels(xticklabels,rotation = 90)
# folder = Path(plotfolder) / 'EventRecords' / 'Stats'
# folder.mkdir(exist_ok=True)
# files = 'signal_metrics_taper_' + str(taper_mode)
# save_tight(folder / (files + '.eps' ),dpi=500)
# # || 77min to complete

In [ ]:
# evaudit = pd.read_pickle(Path(eventsfolder) / 'event_record_audit.pkl')
# evaudit = evaudit[evaudit.nsta>=nsta_bar]
# evsilike = ['2011.242.06.57','2010.246.16.35','2011.191.00.57','2011.236.17.46','2010.163.19.26','2010.096.22.15']
# evsi = [np.where(evaudit.Event==d)[0][0] for d in evsilike]
# evaudit = evaudit.iloc[evsi]

In [ ]:
# ||||||||| Some old HPS Spectrogram Plot Code |||||||||
# ||||||||||||||||||||||||||||||||||||||||||||||||||||||

# # stas = ['X01','M07A','J59C','G03D','M08A'] #I only have time to print out a few. Choose these.
# sta = None
# # sta = "G03A"
# # sta = "M08A"
# # sta = "LA34"
# evireq = None
# # events_folder = 'EVENTS'
# events_folder = 'EVENTS'
# # sta = ['M08A']
# #### -----
# spectrogram_win_len = 200
# bands = [[1/10,1/100],'Spectrogram']
# nrows,ncols = 3,2
# TauP = True
# # bg_tr_color = '#cbc6f4'
# bg_tr_color = '#3266a7'
# Folder = Path(plotfolder) / 'MeetingFigs' / events_folder
# Folder.mkdir(exist_ok=True)
# (Folder / 'HPS_Spectrograms').mkdir(exist_ok=True)
# for (Event,Station,Metrics,Comp) in OBS_Generator(catalog,parent = ATaCR_Py_DataFolder['Py_DataParentFolder'],sta=sta):
#     label = None
#     File = Station.StaName + '.' + '.'.join(Event.split('.')[0:2]) + '.png'
#     OutPath = str(Folder / File)
#     fig, axes = plt.subplots(nrows=nrows, ncols=ncols,figsize=(22,12),height_ratios=[1,1,1],width_ratios=[1,1],layout='constrained',squeeze=False,sharey='col',sharex='row')
#     vmin,vmax = None,None
#     ylabel = True
#     for comp_col,correction_method in enumerate(['HPS']):
#         fn = np.round(fnotch(Station.Water_Depth_m)*1000)/1000
#         for band_row,band in enumerate(bands):
#                 ax = axes[comp_col,band_row]
#                 label = correction_method.replace('ATaCR_HPS','ATaCR & HPS')
#                 if band_row==0:
#                     ttl = '[' + Station.StaName + '], Station Depth: ' + str(Station.Water_Depth_m) + 'm, Notch: ' + str(fn) + 'Hz \n' + Event + ''
#                     fig.suptitle(ttl,fontweight='bold',y=1.06)
#                 if band_row>0:
#                     label=None
#                 if band_row==0:
#                     ax = Metrics['Raw'].plottrace(ax=ax,component='Z',band=band,color=bg_tr_color,lg=False)
#                     ax = Metrics[correction_method].plottrace(ax=ax,component='Z',band=band,color='k',lg=True,st_llaz = Station.STA_LLAZ,ev_llaz=Station.Event_LLAZ,label=label,phases=('P','S','SKS','PKiKP','SKiKS','SKSSKS',))
#                 elif band=='Spectrogram':
#                     if comp_col==2:
#                         pc,ax = Metrics[correction_method].spectrogram(plot=True,r='Z',fig=fig,ax=ax,window=spectrogram_win_len,vmin=vmin,vmax=vmax,ylabel=ylabel)
#                         vmin,vmax = pc.get_clim()
#                     else:
#                         pc,ax = Metrics[correction_method].spectrogram(plot=True,r='Z',fig=fig,ax=ax,window=spectrogram_win_len,vmin=vmin,vmax=vmax,ylabel=ylabel)
#                     if comp_col==0:
#                         vmin,vmax = pc.get_clim()
#                         ylabel = False
#     save_tight(OutPath)
#     plt.close('all')

#     win_length = 200
#     fs = Comp['PostHPS'].stats.sampling_rate
#     (S_full, S_background, S_hps, frequencies, times) = Comp['PostHPS'].spectrograms
#     win_length_samples = int(round(2**np.ceil(np.log2(win_length*fs))))
#     hop_length = win_length_samples // 4
#     n_fft = win_length_samples
#     S_hps, phase = librosa.magphase(librosa.stft(Comp['PostHPS'].data,n_fft=n_fft,hop_length=hop_length,win_length=win_length_samples))
#     hps_spectrograms(S_full, S_background, S_hps, frequencies, times)
#     save_tight(str(Folder / 'HPS_Spectrograms' / File))
#     print('-----')

In [ ]:
# ||||||||| Some old HPS Spectrogram Plot Code |||||||||
# ||||||||||||||||||||||||||||||||||||||||||||||||||||||

# Folder = Path(plotfolder) / 'HPS_Analysis'
# Folder.mkdir(exist_ok=True)
# for (Event,Station,Metrics,Comp) in OBS_Generator(catalog,ATaCR_Py_DataFolder['Py_DataParentFolder']):
# # for i,(Event,Station,Metrics,Comp) in zip(range(1),OBS_Generator(catalog,ATaCR_Py_DataFolder['Py_DataParentFolder'])):
#     File = Station.StaName + '.' + '.'.join(Event.split('.')[0:2]) + '.png'
#     OutPath = str(Folder / File)
#     print(Event)

#     M = Metrics['HPS'] / Metrics['Raw']
#     N = Metrics['Noise']
#     R = Metrics['Raw']
#     A = Metrics['ATaCR'] / Metrics['Raw']
#     pairs_row1 = ['ZP','ZP','1P','2P']
#     pairs_row2 = ['ZZ','ZZ','11','22']
#     pairs_row23 = ['Z','Z','1','2']
#     nrows,ncols = 4,4
#     fig, axes = plt.subplots(nrows=nrows, ncols=ncols,figsize=(25,15),height_ratios=[1,1,1,1],width_ratios=[1,1,1,1],layout='constrained',squeeze=False)
#     # axes = axes.reshape(nrows,ncols)
#     plt.subplots_adjust(wspace=0.05)

#     for col_i,p in enumerate(pairs_row1):
#         row = 0
#         ax = axes[row,col_i]
#         [ax.axvline(f,c='k',alpha=0.3,linewidth=0.5) for f in [1/10,1/30,1/100]]
#         x,y = N.Coherence(p)
#         ax.scatter(x,y,s=0.1,label='Noise | Noise',c='gray')
#         if col_i==0:
#             x,y = A.Coherence(p)
#             ax.plot(x,y,linewidth=0.2,label='Raw | ATaCR',c='r')
#         else:
#             x,y = M.Coherence(p)
#             ax.plot(x,y,linewidth=0.2,label='Raw | HPS',c='b')
#         ax.set_xscale('log')
#         ax.set_xlim(x[1],x[-1])
#         ax.set_ylim(0,1)
#         ax.set_title('[' + p + '] ,Hz',fontweight='bold')
#         ax.legend(loc='upper left',prop={'weight':'bold'})
#         if col_i==0:
#             ax.set_ylabel('Coherence')
#         else:
#             ax.set_yticklabels('')
#         ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
#         ax.set_xticklabels(ax.get_xticklabels(),fontweight='bold')

#     for col_i,p in enumerate(pairs_row2):
#         row = 1
#         ax = axes[row,col_i]
#         [ax.axvline(f,c='k',alpha=0.3,linewidth=0.5) for f in [1/10,1/30,1/100]]
#         if col_i==0:
#             x,y = A.Coherence(p)
#             ax.plot(x,y,c='r',linewidth=0.2,label='Raw | ATaCR')
#         else:
#             x,y = M.Coherence(p)
#             ax.plot(x,y,c='b',linewidth=0.2,label='Raw | HPS')
#         ax.set_xscale('log')
#         ax.set_xlim(x[1],x[-1])
#         ax.set_ylim(0,1)
#         ax.set_title('[' + p + ']',fontweight='bold')
#         ax.set_xticklabels('')
#         ax.legend(loc='upper left',prop={'weight':'bold'})
#         if col_i==0:
#             ax.set_ylabel('Coherence')
#         else:
#             ax.set_yticklabels('')
#     sci_yl = np.floor(np.log10(np.max(np.abs([[np.min(M.traces[c][0].data),np.max(M.traces[c][0].data)] for c in ['Z','1','2']]))))
#     yl_v = np.max(np.abs([[np.min(d.traces['Z'][0].data),np.max(d.traces['Z'][0].data)] for d in [A,M]]))
#     yl_h = np.max([np.max(np.abs([[np.min(d.traces['1'][0].data),np.max(d.traces['1'][0].data)] for d in [A,M]])),np.max(np.abs([[np.min(d.traces['2'][0].data),np.max(d.traces['2'][0].data)] for d in [A,M]]))])
#     for col_i,p in enumerate(pairs_row23):
#         row = 2
#         ax = axes[row,col_i]
#         if col_i==0:
#             tr = A.traces[p][0]
#             clr = 'r'
#             ax.set_title('ATaCR-' + p,fontweight='bold')
#         else:
#             tr = M.traces[p][0]
#             clr = 'b'
#             ax.set_title('HPS-' + p,fontweight='bold')
#         x = R.traces[p][0].times()
#         y = R.traces[p][0].data
#         ax.plot(x,y,label='Raw',linewidth=0.3,c='gray')
#         x = tr.times()
#         y = tr.data
#         ax.plot(x,y,label=tr.stats.location,linewidth=0.5,c=clr)
#         ax.set_xlim(x[0],x[-1])
#         ax.ticklabel_format(axis='both', style='sci', scilimits=(sci_yl,sci_yl+1))
#         ax.set_xticklabels('')
#         ax.legend(loc='upper left',prop={'weight':'bold'})
#         if col_i<=1:
#             ax.set_ylim(-yl_v,yl_v)
#         else:
#             ax.set_ylim(-yl_h,yl_h)
#     for col_i,p in enumerate(pairs_row23):
#         row = 3
#         ax = axes[row,col_i]
#         if col_i==0:
#             f,t,s = A.spectrogram(p,plot=False,window=100,overlap=0.3)
#         else:
#             f,t,s = M.spectrogram(p,plot=False,window=100,overlap=0.3)
#         # s = s - R.spectrogram(p,plot=False,window=100,yscale='log',cmap='nipy_spectral_r')[-1]
#         s = scipy.ndimage.filters.gaussian_filter(s,sigma=0.5,mode='constant')
#         pc = ax.pcolormesh(t,f, 10*np.log10(s), cmap = 'nipy_spectral', shading= 'auto')
#         ax.set_yscale('log')
#         ax.set_ylim(f[1],f[-1])
#         if col_i==(ncols-1):
#             # fig.colorbar(pc,ax=ax, pad=0.01, label='dB')
#             fig.colorbar(pc,ax=ax, pad=0.01, label='dB')
#         if col_i==0:
#             ax.set_ylabel('Frequency Hz')
#         else:
#             ax.set_yticklabels('')
#         ax.set_xticklabels(ax.get_xticklabels(),fontweight='bold')
#         ax.set_xlabel('Time (s)',fontweight='bold')
#         # ax.legend(loc='upper left',prop={'weight':'bold'})
#     ttl = '[' + Station.StaName + '] ' + Event + ' '
#     fig.suptitle(ttl,fontweight='bold',y=1.02)
#     save_tight(OutPath)

In [ ]:
# =======================================================================================================================================================================================================================================
# =======================================================================================================================================================================================================================================